# TP3 - Ejercicio 2
## Máquinas de vectores de soporte

#### Enunciado
a)  Dividir aleatoriamente el conjunto de datos en dos conjuntos, uno de entrenamiento y uno de prueba.

b)  Clasificar la variable categórica _sigdz_ que indica si el paciente posee o no una enfermedad coronaria, utilizando el método SVM. Calcular la matriz de confusión.

c)  Proponer  (y  tambien  realizar)  una  secuencia  de  pasos que  permita  encontrar  una clasificación  mejor  que  la  anterior,  evaluando  diferentes  valores  de C y  diferentes núcleos.

In [5]:
from IPython.display import display
import datetime
import pandas as pd
import numpy as np
import os
from sklearn import svm
from ml_tps.tp3.svm_utils import test_svm_configurations, get_svm_accuracy
from ml_tps.utils.evaluation_utils import getConfusionMatrix, computeAccuracy
from ml_tps.utils.dataframe_utils import divide_in_training_test_datasets, scale_dataset, seperateDatasetObjectiveData

print("Module and package import success")

Module and package import success


In [6]:
dir_path = os.path.dirname(os.path.realpath("e2.ipynb"))
DEFAULT_FILEPATH = f"{dir_path}/../tp3/data/acath.xls"
DEFAULT_OBJECTIVE = "sigdz"
DEFAULT_TRAIN_PCTG = 0.6
DEFAULT_CV_PCTG = 0.2

print("Variable setting and data import success")

Variable setting and data import success


## Información sobre los conjuntos de datos usados

In [7]:
# a)  Divide dataset randomly into training and evaluation set
dataset = pd.read_excel(DEFAULT_FILEPATH)
dataset = dataset.dropna()      # TODO maybe deal with NaN otherwise?
dataset_scaled = scale_dataset(dataset, objective=DEFAULT_OBJECTIVE, ignore_objective=True, scaling_type="minmax")

train, testing_sets = divide_in_training_test_datasets(dataset_scaled, train_pctg=DEFAULT_TRAIN_PCTG)
cv_set, test = divide_in_training_test_datasets(testing_sets, train_pctg=DEFAULT_CV_PCTG/(1-DEFAULT_TRAIN_PCTG))

X_train, y_train = seperateDatasetObjectiveData(train, DEFAULT_OBJECTIVE)
X_cv_set, y_cv_set = seperateDatasetObjectiveData(cv_set, DEFAULT_OBJECTIVE)
X_test, y_test = seperateDatasetObjectiveData(test, DEFAULT_OBJECTIVE)

data_info = pd.Series({"Dimensiones del conjunto de datos original (descartando NaN)": dataset.shape,
                      "Dimensiones del conjunto de entrenamiento": train.shape,
                      "Dimensiones del conjunto de validación cruzada": cv_set.shape,
                      "Dimensiones del conjunto de evaluación": test.shape,
                      "% de datos utilizados para entrenamiento": str(int(DEFAULT_TRAIN_PCTG*100)) + "%",
                      "% de datos utilizados para validación cruzada": str(int(DEFAULT_CV_PCTG*100)) + "%",
                      "% de datos utilizados para evaluación": str(int((1-DEFAULT_TRAIN_PCTG-DEFAULT_CV_PCTG)*100)) + "%",
                      "Variable objetivo de la clasificación": DEFAULT_OBJECTIVE})

pd.DataFrame(data_info, columns=["acath.xls"])

*** No CODEPAGE record, no encoding_override: will use 'ascii'


,acath.xls
Data set dimensions (ignoring NaN),"(2258, 6)"
Training set dimensions,"(1354, 6)"
CV set dimensions,"(452, 6)"
Test set dimensions,"(452, 6)"
% of data set used for training,60%
% of data set used for cross-validation,20%
% of data set used for testing,20%
Classification objective,sigdz


## SVM estándar (b)

In [74]:
# b)  Classify categorical variable "sigdz" using default SVC SVM
words_then = datetime.datetime.now()
c_value1 = 1
kernel1 = "rbf"
clf1 = svm.SVC(kernel=kernel1, gamma='scale', C=c_value1)      # using default parameters, written down for illustrative purposes
clf1.fit(X_train, y_train)

predictions_cv1 = pd.Series(clf1.predict(X_cv_set).T)
confusion_matrix = getConfusionMatrix(predictions_cv1, y_cv_set)
accuracy_train1 = get_svm_accuracy(fitted_classifier=clf1, X=X_train, y=y_train)
accuracy_cv1 = get_svm_accuracy(fitted_classifier=clf1, X=X_cv_set, y=y_cv_set)

data_default_svm = pd.DataFrame(columns=["Núcleo", "Parámetro C", 
                                         "Precisión en conjunto de entrenamiento", 
                                         "Precisión en conjunto de validación cruzada"])
data_default_svm.loc[0] = [kernel1, c_value1, accuracy_train1, accuracy_cv1]

words_now = datetime.datetime.now()
print("Runtime Default SVM fitting and testing: ", divmod((words_now - words_then).total_seconds(), 60), "\n")
print(clf1, "\n\n")

data_default_svm

Runtime Default SVM fitting and testing:  (0.0, 0.091753) 

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False) 




,Núcleo,Parámetro C,Precisión en conjunto de entrenamiento,Precisión en conjunto de validación cruzada
0,rbf,1,0.784343,0.814159


__Matriz de confusión:__

In [75]:
confusion_matrix


Actual,0.0,1.0
Predicted,,
0.0,101,35
1.0,66,250


## Ajuste de parámetros y núcleo (c)

In [76]:
# c)  Evaluate different values for C and different nuclei to find better performing classifiers
kernels = ["rbf", "poly", "linear", "sigmoid"]
c_values = list(np.logspace(-3, 2, 6))
svm_values, best_svm = test_svm_configurations(kernels, c_values, X_train, y_train, X_cv_set, y_cv_set)

time_now = datetime.datetime.now()
print("\n\nRuntime parameter and kernel testing: ", divmod((time_now - words_now).total_seconds(), 60), "\n")

svm_values



Runtime parameter and kernel testing:  (0.0, 2.331971) 



,Kernel,C value,Training set accuracy,CV set accuracy
0,rbf,0.001,0.663220,0.630531
1,rbf,0.010,0.773264,0.750000
2,rbf,0.100,0.773264,0.750000
3,rbf,1.000,0.790990,0.776549
4,rbf,10.000,0.792467,0.767699
5,rbf,100.000,0.799114,0.772124
6,poly,0.001,0.771049,0.750000
7,poly,0.010,0.773264,0.750000
8,poly,0.100,0.773264,0.750000
9,poly,1.000,0.774003,0.750000


### Configuración de SVM con mejor rendimiento:
(en conjunto de validación cruzada)


In [81]:
# Choose SVM with highest accuracy after hyperparameter tuning
winner = svm_values.sort_values(by="CV set accuracy", ascending=False).head(1)
winner

,Kernel,C value,Training set accuracy,CV set accuracy
3,rbf,1.0,0.79099,0.776549


### Precisión en conjunto de evaluación:

In [13]:
best_svm.fit(X_train, y_train)
winner_test_accuracy = get_svm_accuracy(fitted_classifier=best_svm, X=X_test, y=y_test)
    
print("=", winner_test_accuracy)

= 0.7743362831858407
